In [ ]:
import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.llms import AzureOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

from src.hawking_llm import HawkingLLM
from src.text_utils import remove_tags, list_tags, parse_output

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
cko = pd.read_excel('final.xlsx')
cko

,Unnamed: 0,Learn Name,Transcript Link,Media Link,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6 (Learn Page title),...,Summary section of Note (if applicable),SME Name,Book Title,ISBN,chapter number,section title,section link,Transcript text,summary_exists,summary
0,14,Cardiovascular system anatomy and physiology,https://d16qt3wv6xm098.cloudfront.net/dKTQ46dg...,https://d16qt3wv6xm098.cloudfront.net/esdfkE-L...,Organ systems,Cardiovascular system,Physiology,Anatomy and physiology,NaN,Cardiovascular system anatomy and physiology,...,NaN,Tara,Guyton and Hall Textbook of Medical Physiology,9780323597128,9,Cardiac Muscle; The Heart as a Pump and Functi...,https://www.clinicalkey.com/student/content/bo...,_x000D_\nThe circulatory system is also called...,True,"_x000D_\nAll right, as a quick recap, the pulm..."
1,15,Lymphatic system anatomy and physiology,https://d16qt3wv6xm098.cloudfront.net/kUI5Rpyb...,https://d16qt3wv6xm098.cloudfront.net/PFyWFaLs...,Organ systems,Cardiovascular system,Physiology,Anatomy and physiology,NaN,Lymphatic system anatomy and physiology,...,NaN,Tara,Gray’s Anatomy for Students,9780323393041,1,Lymphatic system,https://www.clinicalkey.com/student/content/bo...,"“Lymph” means “clear water” in Latin, and it d...",True,"_x000D_\nAll right, as a quick recap: the lymp..."
2,17,"Blood pressure, blood flow, and resistance",https://d16qt3wv6xm098.cloudfront.net/VoaTD7vM...,https://d16qt3wv6xm098.cloudfront.net/f8VmVzlG...,Organ systems,Cardiovascular system,Physiology,Hemodynamics,Principles of hemodynamics,"Blood pressure, blood flow, and resistance",...,NaN,Tara,Costanzo Physiology,9780323793339,4,Velocity of blood flow,https://www.clinicalkey.com/student/content/bo...,"Pressure is a force over an area, so with bloo...",True,"_x000D_\nAll right, as a quick recap: blood pr..."
3,18,Pressures in the cardiovascular system,https://d16qt3wv6xm098.cloudfront.net/jUyNoNCp...,https://d16qt3wv6xm098.cloudfront.net/dDbM5yhb...,Organ systems,Cardiovascular system,Physiology,Hemodynamics,Principles of hemodynamics,Pressures in the cardiovascular system,...,NaN,Tara,Costanzo Physiology,9780323793339,4,Arterial pressure in the systemic circulation,https://www.clinicalkey.com/student/content/bo...,When we talk about pressures in the cardiovasc...,True,"\nAll right, as a quick recap: vascular pressu..."
4,19,Laminar flow and Reynolds number,https://d16qt3wv6xm098.cloudfront.net/DDpPLdBP...,https://d16qt3wv6xm098.cloudfront.net/CCRL1WtY...,Organ systems,Cardiovascular system,Physiology,Hemodynamics,Principles of hemodynamics,Laminar flow and Reynolds number,...,NaN,Tara,Costanzo Physiology,9780323793339,4,Laminar flow and reynolds number,https://www.clinicalkey.com/student/content/bo...,"Laminar means smooth, and so laminar blood flo...",True,"\nAll right, as a quick recap: The Reynold’s n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,130,Pericarditis and pericardial effusion,https://d16qt3wv6xm098.cloudfront.net/2iN0zIp7...,NaN,Organ systems,Cardiovascular system,Pathology,Pericardial disorders,NaN,Pericarditis and pericardial effusion,...,NaN,Tara,Robbins & Kumar Basic Pathology,9780323790185,9,Pericardial Effusion and Hemopericardium,https://www.clinicalkey.com/student/content/bo...,"With pericarditis, “peri” means “around,” card...",True,"_x000D_\nOkay, let’s recap - acute pericarditi..."
58,131,Cardiac tamponade,https://d16qt3wv6xm098.cloudfront.net/-P2N4R1-...,NaN,Organ systems,Cardiovascular system,Pathology,Pericardial disorders,NaN,Cardiac tamponade,...,NaN,Tara,Robbins & Kumar Basic Pathology,9780323790185,9,Pericardial Effusion and Hemopericardium,https://www.clinicalkey.com/student/content/bo...,The name “cardiac tamponade” can be broken dow...,True,"_x000D_\nAll right, as a quick recap… Cardiac ..."
59,133,Cardiac tumors,https://d16qt3wv6xm098.cloudfront.net/lMnh61-0...,NaN,Organ systems,Cardiovascular system,Pathology,Cardiac tumors,NaN,Cardiac tumors,...,NaN,Tara,

In [ ]:
import pandas as pd

# Assuming your DataFrame is named 'df'
cko['Transcript text'] = cko['Transcript text'].str.replace('_x000D_\n', '').str.replace('\n', '')

# Get the transcripts as a list
transcripts = cko['Transcript text'].tolist()

In [ ]:
# Assuming your DataFrame is named 'df'
cko['summary'] = cko['summary'].str.replace('_x000D_\n', '').str.replace('\n', '')

# Get the transcripts as a list
reference_summaries = cko['summary'].tolist()

In [ ]:
print(len(transcripts))
print(len(reference_summaries))

62
62


In [ ]:
reference_summaries[4]

'All right, as a quick recap: The Reynold’s number is written as Re = density times diameter times velocity over viscosity, and is a way to predict when fluid’s going to be either laminar or turbulent. In general, Reynold’s numbers below 2000 will have laminar flow, and Reynolds numbers above 3000 will have turbulent flow. Furthermore, differences in velocity across a blood vessel causes shear.'

In [ ]:
transcripts[0]

'The circulatory system is also called the cardiovascular system, where “cardi” refers to the heart, and “vascular” refers to the blood vessels. So, these are the two key parts: the heart, which pumps blood, and the blood vessels, which carry blood to the body and return it back to the heart again. Ultimately, this is how nutrients like O2, or oxygen, get pushed out to the organs and tissues that need it, and how waste like CO2, or carbon dioxide, which is the main byproduct of cellular respiration, gets removed.The heart is about the size of a person’s fist, which makes sense: a bigger person has a bigger fist and, therefore, a bigger heart.And it’s shaped like a cone, and sits slightly shifted over to the left side, in the mediastinum, which is the middle of the chest cavity, or thorax.It sits on top of the diaphragm, which is the main muscle that helps with breathing, behind the sternum, or breastbone, in front of the vertebral column, squished in between the two lungs, and protecte

In [ ]:
#preprocessing to remove ###SUMMARY from each transcript

new_transcripts = []
for transcript in transcripts:
    summary_start = transcript.find("###SUMMARY")
    if summary_start != -1:
        new_transcript = transcript[:summary_start]
    else:
        new_transcript = transcript
    new_transcripts.append(new_transcript.strip())

# Print the new transcripts
for new_transcript in new_transcripts:
    print(new_transcript)


The circulatory system is also called the cardiovascular system, where “cardi” refers to the heart, and “vascular” refers to the blood vessels. So, these are the two key parts: the heart, which pumps blood, and the blood vessels, which carry blood to the body and return it back to the heart again. Ultimately, this is how nutrients like O2, or oxygen, get pushed out to the organs and tissues that need it, and how waste like CO2, or carbon dioxide, which is the main byproduct of cellular respiration, gets removed.The heart is about the size of a person’s fist, which makes sense: a bigger person has a bigger fist and, therefore, a bigger heart.And it’s shaped like a cone, and sits slightly shifted over to the left side, in the mediastinum, which is the middle of the chest cavity, or thorax.It sits on top of the diaphragm, which is the main muscle that helps with breathing, behind the sternum, or breastbone, in front of the vertebral column, squished in between the two lungs, and protected

In [ ]:
new_transcripts[4]

"Laminar means smooth, and so laminar blood flow is blood that’s flowing smoothly through the vessels. Turbulent flow, on the other hand, is when the blood’s not flowing smoothly, and we can figure out if blood is likely to be laminar or turbulent by finding its Reynolds number or Re, which is named after Osborne Reynolds, a Victorian scientist who not only studied fluid dynamics, but is a man that knows how to rock a beard and bowtie.If everything’s moving like it should and the blood flow is laminar, the linear velocity of the blood  -- how fast it’s moving in a straight line -- is greatest in the center of the blood vessel, and lowest near the walls of the vessel, dropping to zero at the wall. Sometimes, though, blood flow is disrupted, like if it has to pass by a crusty old atherosclerotic plaque along the wall, which reduces the diameter of the blood vessel at that point and causes turbulence. There are a number of factors help predict turbulence, they include the density of the b

In [ ]:
transcripts[4]

"Laminar means smooth, and so laminar blood flow is blood that’s flowing smoothly through the vessels. Turbulent flow, on the other hand, is when the blood’s not flowing smoothly, and we can figure out if blood is likely to be laminar or turbulent by finding its Reynolds number or Re, which is named after Osborne Reynolds, a Victorian scientist who not only studied fluid dynamics, but is a man that knows how to rock a beard and bowtie.If everything’s moving like it should and the blood flow is laminar, the linear velocity of the blood  -- how fast it’s moving in a straight line -- is greatest in the center of the blood vessel, and lowest near the walls of the vessel, dropping to zero at the wall. Sometimes, though, blood flow is disrupted, like if it has to pass by a crusty old atherosclerotic plaque along the wall, which reduces the diameter of the blood vessel at that point and causes turbulence. There are a number of factors help predict turbulence, they include the density of the b

In [ ]:
# Create connector and set max tokens per response
vicuna = HawkingLLM(max_new_tokens=512, temperature = 0.01)

# Test the model
vicuna('Explain nuclear physics in no more than five sentences.')

'\n\nNuclear physics is the branch of physics that studies the behavior and properties of atomic nuclei, which are made up of protons and neutrons. The nucleus of an atom contains a large amount of positive charge, or protons, which repel each other. This force keeps the nucleus stable, even though it is made up of positively charged particles. Nuclear reactions occur when two atoms collide and exchange particles, such as protons or neutrons. These reactions can release energy in the form of light or heat, and they power everything from stars to nuclear reactors. However, nuclear reactions also have the potential to be dangerous, as seen in events like nuclear bombs or accidents at nuclear power plants.'

In [ ]:
import requests
import xml.etree.ElementTree as ET
#from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KhandelwalR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KhandelwalR\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#create a prompt template

template = (
    """
    {context}

    Please summarize the above text. The summary should be concise and capture all the main findings and clinical terms.

    """
)
prompt_template = PromptTemplate.from_template(template)
print(prompt_template.input_variables)

['context']


In [ ]:
# rough

#output_parser = CommaSeparatedListOutputParser()

#format_instructions = output_parser.get_format_instructions()

prompt = prompt_template.format(
    context=transcripts[1]
    #format_instructions = format_instructions
)

print(prompt)

output = vicuna(prompt)

print(output)

In [ ]:
#rough prompt
A chat between a curious user and a clinical specialist. The clinical specialist gives concise answers to the user's questions.

    USER:
    {context}

    Summarize the above text.

    Clinical Specialist:

In [ ]:
from rouge_score import rouge_scorer


In [ ]:
#ONLY NEED TO RUN FOR THE FIRST PROMPT. THEREAFTER JUST LOAD THE DICTIONARY
# creating a dictionary to store the summaries generated by each prompt
gen_sum = {}
for i in range(80):
    gen_sum[i] = {}
    for j in range(62):
        gen_sum[i][j] = []

In [ ]:
gen_sum

{0: {0: [],
  1: [],
  2: [],
  3: [],
  4: [],
  5: [],
  6: [],
  7: [],
  8: [],
  9: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [],
  15: [],
  16: [],
  17: [],
  18: [],
  19: [],
  20: [],
  21: [],
  22: [],
  23: [],
  24: [],
  25: [],
  26: [],
  27: [],
  28: [],
  29: [],
  30: [],
  31: [],
  32: [],
  33: [],
  34: [],
  35: [],
  36: [],
  37: [],
  38: [],
  39: [],
  40: [],
  41: [],
  42: [],
  43: [],
  44: [],
  45: [],
  46: [],
  47: [],
  48: [],
  49: [],
  50: [],
  51: [],
  52: [],
  53: [],
  54: [],
  55: [],
  56: [],
  57: [],
  58: [],
  59: [],
  60: [],
  61: []},
 1: {0: [],
  1: [],
  2: [],
  3: [],
  4: [],
  5: [],
  6: [],
  7: [],
  8: [],
  9: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: [],
  15: [],
  16: [],
  17: [],
  18: [],
  19: [],
  20: [],
  21: [],
  22: [],
  23: [],
  24: [],
  25: [],
  26: [],
  27: [],
  28: [],
  29: [],
  30: [],
  31: [],
  32: [],
  33: [],
  34: [],
  35: [],
  36: [],
  37: [],
  38: [],
  3

In [ ]:
#rough
for i, note in enumerate(new_transcripts):
    gen_sum[0][i].append(note)

In [ ]:
#rough
gen_sum

{0: {0: ['The circulatory system is also called the cardiovascular system, where “cardi” refers to the heart, and “vascular” refers to the blood vessels. So, these are the two key parts: the heart, which pumps blood, and the blood vessels, which carry blood to the body and return it back to the heart again. Ultimately, this is how nutrients like O2, or oxygen, get pushed out to the organs and tissues that need it, and how waste like CO2, or carbon dioxide, which is the main byproduct of cellular respiration, gets removed.The heart is about the size of a person’s fist, which makes sense: a bigger person has a bigger fist and, therefore, a bigger heart.And it’s shaped like a cone, and sits slightly shifted over to the left side, in the mediastinum, which is the middle of the chest cavity, or thorax.It sits on top of the diaphragm, which is the main muscle that helps with breathing, behind the sternum, or breastbone, in front of the vertebral column, squished in between the two lungs, and

In [ ]:
import pickle

# Load the dictionary from the file
with open("gen_sum_20.pkl", "rb") as file:
    gen_sum = pickle.load(file)

In [ ]:
gen_sum

{0: {0: ['\n        The text describes the function of the heart and blood vessels, including how the heart pumps blood and how blood flows throughout the body. It discusses the differences between arteries and veins, including their respective volumes and pressures. Additionally, it explains the structure of blood vessels, including the three layers that surround the vessel lumen, and how blood vessels adapt to changes in blood flow.'],
  1: ["\n\nThe lymphatic system is responsible for maintaining proper fluid balance within the body, transporting lymph from the interstitial space to the bloodstream, and filtering out waste products and harmful substances. It includes various components such as lymphatic vessels, lymph nodes, lacteals, Peyer's patches, spleen, and thymus. Each component performs specific functions related to the overall health of the body."],
  2: ['\n        * Blood pressure is the force applied to the walls of blood vessels by the blood itself.\n        \n        *

In [ ]:
#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize osmosis transcripts and evaluate
for i, note in enumerate(new_transcripts):

    prompt = prompt_template.format(
        context=note
    )
    summary = vicuna(prompt)

    gen_sum[20][i].append(summary)      ##########################update##########################

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[reference_summaries[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = reference_summaries[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    print("Transcript:")
    print(note)
    print("\nReference Summary:")
    print(reference_summaries[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")


C:\Users\KhandelwalR\AppData\Local\anaconda3\envs\LGLLM\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\KhandelwalR\AppData\Local\anaconda3\envs\LGLLM\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Transcript:
The circulatory system is also called the cardiovascular system, where “cardi” refers to the heart, and “vascular” refers to the blood vessels. So, these are the two key parts: the heart, which pumps blood, and the blood vessels, which carry blood to the body and return it back to the heart again. Ultimately, this is how nutrients like O2, or oxygen, get pushed out to the organs and tissues that need it, and how waste like CO2, or carbon dioxide, which is the main byproduct of cellular respiration, gets removed.The heart is about the size of a person’s fist, which makes sense: a bigger person has a bigger fist and, therefore, a bigger heart.And it’s shaped like a cone, and sits slightly shifted over to the left side, in the mediastinum, which is the middle of the chest cavity, or thorax.It sits on top of the diaphragm, which is the main muscle that helps with breathing, behind the sternum, or breastbone, in front of the vertebral column, squished in between the two lungs, a

Transcript:
“Lymph” means “clear water” in Latin, and it describes the fluid that flows through the lymphatic vessels and lymph nodes which make up the lymphatic system. The three major roles of the lymphatic system - the reason we need it in the first place - are that it returns fluid from the tissues back to the heart, it helps large molecules like hormones and lipids enter the blood, and it helps with immune surveillance to keep infections from running amok.So, let’s take a closer look at lymph and where it comes from. The blood in the arteries is under a lot of pressure because it needs to reach every little nook and cranny of the body. The arteries branch out into narrower and narrower arteries, and then arterioles, and finally gets to the capillaries - which have walls that are only one cell thick and are slightly porous. Red blood cells are too big to fit through capillary pores, but small proteins like albumin and fluid can make it through. Every day 20 liters of fluid water an

Transcript:
Pressure is a force over an area, so with blood pressure, we’re measuring the force that the blood exerts on the surface area of the walls of the blood vessels. Differences in blood pressure throughout the body keep blood flowing from high-pressure areas, like the arteries, to low-pressure areas, like the veins. When we say “blood flow,” we’re referring to the volume of blood that flows through a vessel or an organ over some period of time. Now, the amount of blood flow from one end of a blood vessel to another is affected by the blood pressure, and by the resistance, which comes from the vessels themselves. Vasoconstriction, where the vessels constrict, decreases blood flow, and vasodilation, where the blood vessels expand, increases blood flow.   Now, blood flow is not the same thing as the velocity of blood. Blood flow is the volume of blood that moves by a point over some period of time. So let’s say this chunk of blood has a volume of 83 cm^3, and it took 1 second for 

Transcript:
Laminar means smooth, and so laminar blood flow is blood that’s flowing smoothly through the vessels. Turbulent flow, on the other hand, is when the blood’s not flowing smoothly, and we can figure out if blood is likely to be laminar or turbulent by finding its Reynolds number or Re, which is named after Osborne Reynolds, a Victorian scientist who not only studied fluid dynamics, but is a man that knows how to rock a beard and bowtie.If everything’s moving like it should and the blood flow is laminar, the linear velocity of the blood  -- how fast it’s moving in a straight line -- is greatest in the center of the blood vessel, and lowest near the walls of the vessel, dropping to zero at the wall. Sometimes, though, blood flow is disrupted, like if it has to pass by a crusty old atherosclerotic plaque along the wall, which reduces the diameter of the blood vessel at that point and causes turbulence. There are a number of factors help predict turbulence, they include the densi

Transcript:
Compliance, which is sometimes called capacitance or distensibility, refers to the ability of a vessel to respond to an increase in pressure by to distending or swell and increase the volume of blood it can hold, or with decreased pressure, a decrease in volume. The way that this applies to blood vessels is to remember that they are stretchable tubes like rubber hoses rather than lead pipes. So if the pressure increases, the walls of the tube can actually stretch out a bit to accommodate a larger volume, and exactly how much they stretch out depends on their compliance.We can calculate a given blood vessel’s compliance, C, by dividing the volume of blood, V, in mL by the amount of pressure (P) in mmHg, that the blood is experiencing. And so we measure compliance in mL / mm Hg.So we can plot out volume as a function of pressure, where the slope, volume over pressure, is the compliance. The veins have high compliance, meaning they’re high-volume, low pressure vessels, and eve

Transcript:
More than a century ago, two physiologists, Otto Frank and Ernest Starling discovered that as the heart gets filled up with more blood during diastole, it contracts harder and pumps out more blood during systole. So they came up with the Frank Starling Law to explain this relationship. To understand this relationship, let’s zoom into the wall of the ventricles. The bulk of these walls is made up of short, branched cardiac muscle cells packed one next to the other. Zooming in further, if we look inside the muscle cells, we see bundles of myofibrils, or long chains of sarcomeres. The sarcomere is the smallest structure in the muscle that is capable of contracting so it's considered the basic contractile unit of the muscle. The sarcomere has two Z discs that form its boundary and a M line in the middle.  Attached to the Z disc are thin filaments made of actin protein. These actin filaments have structural polarity which means both ends of the filament look different.  We can t

In [ ]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

In [ ]:
gen_sum

{0: {0: ['\n        The text describes the function of the heart and blood vessels, including how the heart pumps blood and how blood flows throughout the body. It discusses the differences between arteries and veins, including their respective volumes and pressures. Additionally, it explains the structure of blood vessels, including the three layers that surround the vessel lumen, and how blood vessels adapt to changes in blood flow.'],
  1: ["\n\nThe lymphatic system is responsible for maintaining proper fluid balance within the body, transporting lymph from the interstitial space to the bloodstream, and filtering out waste products and harmful substances. It includes various components such as lymphatic vessels, lymph nodes, lacteals, Peyer's patches, spleen, and thymus. Each component performs specific functions related to the overall health of the body."],
  2: ['\n        * Blood pressure is the force applied to the walls of blood vessels by the blood itself.\n        \n        *

In [ ]:
#rough

for i, note in enumerate(new_transcripts):
    gen_sum[4][i] = []

In [ ]:
import pickle

# Save the dictionary to a file
with open("gen_sum_21.pkl", "wb") as file:
    pickle.dump(gen_sum, file)

In [ ]:
import pickle

# Load the dictionary from the file
with open("gen_sum_10.pkl", "rb") as file:
    gen_sum = pickle.load(file)

In [ ]:
gen_sum

{0: {0: ['\n        The text describes the function of the heart and blood vessels, including how the heart pumps blood and how blood flows throughout the body. It discusses the differences between arteries and veins, including their respective volumes and pressures. Additionally, it explains the structure of blood vessels, including the three layers that surround the vessel lumen, and how blood vessels adapt to changes in blood flow.'],
  1: ["\n\nThe lymphatic system is responsible for maintaining proper fluid balance within the body, transporting lymph from the interstitial space to the bloodstream, and filtering out waste products and harmful substances. It includes various components such as lymphatic vessels, lymph nodes, lacteals, Peyer's patches, spleen, and thymus. Each component performs specific functions related to the overall health of the body."],
  2: ['\n        * Blood pressure is the force applied to the walls of blood vessels by the blood itself.\n        \n        *